라이브러리
===========

In [ ]:
import cv2
import json
import csv
import os
import numpy as np
import re
import shutil
import glob
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

압축해제
======================

In [ ]:
#압축 해제코드 (드라이브에 업로드하는 파일이 많을 경우 멈추는 거 대비)
#-o:자동 덮어쓰기 -d:새로운 경로에 압축해제
!unzip /content/drive/MyDrive/newstudy/fire_segmentation/train.zip -d /content/drive/MyDrive/newstudy/fire_segmentation/train

Segmentation 데이터 준비 과정
======================

In [ ]:

dataset_dir = '/content/drive/MyDrive/newstudy/fire_segmentation/train'
data_dir_images = dataset_dir+'/images'
data_dir_masks = dataset_dir+'/masks'

with open(dataset_dir+'/labels.json') as f:
  data = json.load(f)

print(type(data))
print(data.keys())

#배열로 전환해서 데이터 저장 형식 확인
data2 = np.array(data['annotations'])
print(type(data2))
print(data2)

In [ ]:
all = len(os.listdir(data_dir_images))
print('개수:',all)

fire_segment(1c)
==================

In [ ]:


'''
class
0 : 검은색 연기
1 : 회색 연기
2 : 하얀 연기
3 : 화재
'''

if os.path.exists(data_dir_masks)== False:
  os.makedirs(data_dir_masks)

success=0
failed = []

for i in range(len(data['annotations'])):
  filename = data['annotations'][i]['file_name']
  if os.path.exists(data_dir_images + '/'+filename):
    img = cv2.imread(data_dir_images + '/'+filename)

    #BGR형식에서 RGB 형식을 전환 후 검은화면으로 만들어주기
    mask = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask[mask>0] = 0

    #클래스별 도형 만들어주기
    for j in range(len(data['annotations'][i]['objects'])):
      poly = data['annotations'][i]['objects'][j]['polygon']
      poly = np.array(poly)
      mask = cv2.fillPoly(mask, np.int32([poly]), color = data['annotations'][i]['objects'][j]['class'])
      print(str(data['annotations'][i]['objects'][j]['class'])+'클래스')
    
    mask = np.uint8(mask)

    #파일의 정보표시 후 저장 (성공 개수/실행 횟수/총 개수) jpg로 저장시 이미지에 변화가 생김 ex) 0,0,255 -> 24,24,255
    print(f'저장 중:{i}.png-{mask.shape}-({success}/{i}/'+str(len(data['annotations']))+')')
    cv2.imwrite(data_dir_masks +'/' + str(i)+'.png', mask)

    success += 1
  else :
    failed.append(failed)

print(f'총 {all}개 중 {success}개가 저장 되었습니다.')

if len(failed):
  print(f'{len(failed)}개의 마스크를 만드는데 실패하였습니다.')
  print(failed)